# Neptune + Prophet

## Introduction

This guide will show you how to log the following Prophet metadata with Neptune, using a single function:

* Regression summary metadata
* Input and forecast Dataframes
* Forecast, components and residual diagnostics plots
* Model configuration
* Serialized model

[![image](https://neptune.ai/wp-content/uploads/fbprophet-neptune-dashboard.png)](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/)

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [1]:
! pip install -U neptune-prophet prophet

  Using cached matplotlib-3.4.3-cp38-cp38-win_amd64.whl (7.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.0
    Uninstalling matplotlib-3.7.0:
      Successfully uninstalled matplotlib-3.7.0
  Attempting uninstall: neptune-prophet
    Found existing installation: neptune-prophet 0.4.1
    Uninstalling neptune-prophet-0.4.1:
      Successfully uninstalled neptune-prophet-0.4.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
neptune-xgboost 1.0.0 requires neptune-client>=0.16.17, which is not installed.
neptune-sklearn 2.0.0 requires neptune-client>=0.16.17, which is not installed.
neptune-lightgbm 1.0.0 requires neptune-client>=0.16.17, which is not installed.


In [2]:
import neptune

## Initialize Neptune and create new run
Connect your script to Neptune application and create a new run.

In [3]:
run = neptune.init_run(
    project="common/fbprophet-integration",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags=["prophet", "artifacts", "notebook"],  # optional
)

c:\Users\siddh\miniconda3\envs\py38\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


https://app.neptune.ai/common/fbprophet-integration/e/FBPROP-3034


Click on the link above to open this run in Neptune. For now it is empty, but keep the tab with run open to see what happens next.

With the `api_token` and `project` arguments, you tell Neptune: 

* **who you are**: with your Neptune API token
* **where you want to send your data**: your Neptune project

At this point you have new run in Neptune. From now on, we'll use the `run` object to log metadata.

---

**Note**


Instead of logging data to the public project `'common/fbprophet-integration'` as an anonymous user, you can log it to your own project.

To find and copy your credentials:

1. In the Neptune app, click your avatar and select **Get your API token**.
2. Pass the token to ``api_token`` argument of the ``init_run()`` method.
3. Pass your project name to the ``project`` argument. You can find it in the **project settings** &rarr; **Properties**.

For example:

```python
neptune.init_run(
    project="ml-team/classification",
    api_token="h0dHBzOi8aHR0cHM6Lkc78ghs74kl0jvYh3Kb8",
)
```

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_R.csv"
)

# Market capacity
df["cap"] = 8.5

In [6]:
from prophet import Prophet

In [7]:
model = Prophet(growth="logistic")
model.fit(df)

13:48:41 - cmdstanpy - INFO - Chain [1] start processing
13:48:42 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
forecast = model.predict(df)

### Log summary metadata

Log the Prophet regression summary to Neptune using the `create_summary()` function:


In [9]:
import neptune.new.integrations.prophet as npt_utils

run["prophet_summary"] = npt_utils.create_summary(model=model, df=df, fcst=forecast)

c:\Users\siddh\miniconda3\envs\py38\lib\site-packages\neptune\common\warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  warnings.warn(


## Stop logging

Once you are done logging, stop tracking the run.

In [10]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/common/fbprophet-integration/e/FBPROP-3034/metadata


### Explore results in Neptune

You just learned how to log the Prophet regressor summary to Neptune using a single function. 

Click on the link that was printed to the console or [go here](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdiagnostics_charts%2F) to explore a run similar to yours. In particular check:

* [logged dataframes](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdataframes%2F)
* [logged regressor visualizations](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fdiagnostics_charts%2F)
* [logged model config](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fmodel%2Fmodel_config%2F)
* [logged serialized model](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=prophet_summary%2Fmodel%2F&attribute=serialized_model)
* [logged metadata](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/all?path=sys)
* [logged code and git metadata](https://app.neptune.ai/o/common/org/fbprophet-integration/e/FBPROP-249/source-code?path=source_code&file=Neptune_prophet.py)

## More Options

The Neptune + Prophet integration also lets you log only specific metadata of your choice, by using additional methods.
Below are a few examples.

For the full example, see the [Neptune docs](https://docs.neptune.ai/integrations-and-supported-tools/model-training/prophet).

### Before you start: create, fit and predict a Prophet model

In [11]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_R.csv"
)

# Market capacity
df["cap"] = 8.5

model = Prophet(growth="logistic")
model.fit(df)
forecast = model.predict(df)

13:56:08 - cmdstanpy - INFO - Chain [1] start processing
13:56:08 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
import neptune.integrations.prophet as npt_utils

run = neptune.init_run(
    project="common/fbprophet-integration",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags=["prophet", "artifacts", "notebook"],  # optional
)

https://app.neptune.ai/common/fbprophet-integration/e/FBPROP-3037


Open the link above to see the metadata logging results, as we add them below.

### Log Prophet plots

In [13]:
run["forecast_plots"] = npt_utils.create_forecast_plots(model, forecast)

In [14]:
run["forecast_components"] = npt_utils.get_forecast_components(model, forecast)

In [15]:
run["residual_diagnostics_plot"] = npt_utils.create_residual_diagnostics_plots(forecast, df.y)

### Log Prophet model configuration 

In [16]:
run["model_config"] = npt_utils.get_model_config(model)

### Log Prophet serialized model

In [17]:
run["model"] = npt_utils.get_serialized_model(model)

## Stop logging

Once you are done logging, stop tracking the run.

In [18]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 39 operations to synchronize with Neptune. Do not kill this process.
All 39 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/common/fbprophet-integration/e/FBPROP-3037/metadata
